In [1]:
#Bootstrapping
import pandas as pd
import numpy as np

df=pd.read_csv('/home/dherya.malhotra/Carpe_DM/PAPER_CODES/MBZUAI_POC_PAPER_DATA - Sheet1.csv')
df=df[df['Stock market return (%, year-on-year)(Y)']!=0]
df = df[df['Oil Export Price (X)($/Barrel)'].notna()]
df[df['Country']=='UAE']

,Country,Crude Grade,Year,Oil Export Price (X)($/Barrel),"Stock market return (%, year-on-year)(Y)",GDP per Capita (Constant 2015 US$)(Z),Political Risk Index(Z)($)
11,UAE,Murban,2011,109.77,14.823190,43800.15930,0.912176
12,UAE,Murban,2012,111.76,-6.251017,42868.09750,0.863251
13,UAE,Murban,2013,108.21,-6.519889,43269.84047,0.894792
14,UAE,Murban,2014,99.45,4.275693,43023.19042,0.768909
15,UAE,Murban,2015,53.87,0.309401,43534.99727,0.748049
16,UAE,Murban,2016,44.83,1.552132,43745.28817,0.549646
17,UAE,Murban,2017,54.82,8.972898,42690.21478,0.601950
18,UAE,Murban,2018,72.20,2.061658,42679.75191,0.689256
19,UAE,Murban,2019,64.72,-13.216550,42700.12979,0.667538
20,UAE,Murban,2020,42.98,24.706250,40776.63087,0.592605


In [2]:
A=df[["Oil Export Price (X)($/Barrel)","Stock market return (%, year-on-year)(Y)","GDP per Capita (Constant 2015 US$)(Z)","Political Risk Index(Z)($)"]]
A.shape

(110, 4)

In [3]:

def Bootstrap(A):
    r=A.shape[0]
    c=A.shape[1]
    ans=pd.DataFrame(columns=A.columns)
    for i in range(r):
        temp=np.random.randint(0,r)
        ans.loc[len(ans)]=A.iloc[temp]
    return ans

In [4]:
def erf_asy(x):
    a1, a2, a3, a4, a5 = 0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429
    p = 0.3275911

    sign = np.sign(x)  # Element-wise sign function
    x = np.abs(x)
    t = 1 / (1 + p * x)
    y = 1 - (((((a5 * t + a4) * t + a3) * t + a2) * t + a1) * t * np.exp(-x * x))
    return sign * y



In [5]:
#P(Ya<y<Yb|Y=y',X=x') under monotonicity assumption
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm


from scipy.integrate import quad



def PNS_Yx_Bootstrap_CI(A, iter, alpha, y,y_,a,b,x_,c1,c2):

    n = A.shape[0]
    PNS_Y0_estimate = []
    threshold=[]
    for j in range(iter):
        B = Bootstrap(A) 
        II_y = (B['Stock market return (%, year-on-year)(Y)'] < y).astype(int)
        II_y_= (B['Stock market return (%, year-on-year)(Y)'] < y_).astype(int)

        X = np.array(B['Oil Export Price (X)($/Barrel)']).reshape(-1, 1)
        C1 = np.array(B['GDP per Capita (Constant 2015 US$)(Z)']).reshape(-1, 1)
        C2 = np.array(B['Political Risk Index(Z)($)']).reshape(-1, 1)
        clf = LogisticRegression(max_iter=500)
        clf.fit(np.hstack([X, C1, C2]), II_y)
        clf_1 = LogisticRegression(max_iter=500)
        clf_1.fit(np.hstack([X, C1, C2]), II_y_)
        P0_y = clf.predict_proba(np.array([[a,c1,c2]]))[:, 1]
        P1_y = clf.predict_proba(np.array([[b,c1,c2]]))[:, 1]
        Px__y_=clf_1.predict_proba(np.array([[x_,c1,c2]]))[:, 1]
        temp=0
        if((P1_y<Px__y_) and (Px__y_<P0_y)):
            temp=1
        PNS_Y0_estimate.append(temp)
    PNS_Y0_estimate = np.array(PNS_Y0_estimate)
    lower = np.percentile(PNS_Y0_estimate, alpha)
    upper = np.percentile(PNS_Y0_estimate, 100 - alpha)
    mean = np.mean(PNS_Y0_estimate)
    return np.array([lower, mean, upper])


In [6]:


row = df[(df['Country'] == 'UAE') & (df['Year'] == 2014)].iloc[0]

y_ = float(row['Stock market return (%, year-on-year)(Y)'])
b = float(row['Oil Export Price (X)($/Barrel)'])
c1=float(row['GDP per Capita (Constant 2015 US$)(Z)'])
c2=float(row['Political Risk Index(Z)($)'])

In [7]:
y=0
a=80
x_=b
temp1=PNS_Yx_Bootstrap_CI(A, 100, 2.5, y,y_,a,b,x_,c1,c2)

In [8]:
print('PNS_evidence_under_monotonicity_CI',temp1)

PNS_evidence_under_monotonicity_CI [0.   0.76 1.  ]
